In [1]:
from groq import Groq
import pandas as pd
import re
import os

In [5]:
df = pd.read_csv('../../data_acquisition/data/df_with_salary.csv')

In [10]:
df.head()

,title,company,contract_type,description,min_workload,max_workload,min_salary,max_salary,block1_2_min_salary,block1_2_max_salary,block1_2_answer
0,Software Engineer .Net,ELCA Informatik AG,Festanstellung,"Über ELCA Wir sind ELCA, einer der grössten Sc...",80,100,100000,130000,None,None,None
1,Digital Analyst / Web Analyst,Unic AG,Festanstellung,Deine Aufgaben als Digital Analyst bei Unic ...,80,100,90000,113000,None,None,None
2,ICT Supporter 100% (a),Spitex Zürich,Festanstellung,Möchtest du auch etwas bewegen und deine Arbei...,100,100,78000,96000,None,None,None
3,KAUFMÄNNISCHES PRAKTIKUM (100%) - MIT FOKUS KU...,gebana AG,Praktikum,DEINE AUFGABEN IM KUND:INNENSERVICE – EINSATZ ...,100,100,28300,28300,None,None,None
4,System Engineer für Microsoft-Technologien und...,konekkt GmbH,Festanstellung,Wir präsentieren eine faszinierende Karrieremö...,80,100,95000,125000,None,None,None


Block 0: Few shot 5 examples with salary 
Block 1: Job title 
Block 2: company 
Block 3: workload (min workload and max workload) 
Block 4: Contract type 
Block 5: description

# Set up for Usage

In [8]:
client = Groq(
    api_key="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",
)

def generate_message(title, company, min_salary, max_salary, min_workload, max_workload, description):
    return {
        "role": "user",
        "content": f"Das Gehalt für die Position {title} bei der Firma {company} liegt zwischen {min_salary} und {max_salary} mit einer Arbeitszeit von {min_workload} bis {max_workload} Prozent. Hier ist die Beschreibung, was Sie benötigen und tun müssen: {description}"
    }

def extract_salaries(response):
    pattern = re.compile(r"(\d{1,3}(?:['\s]\d{3})*)\s*(?:-|und)\s*(\d{1,3}(?:['\s]\d{3})*)\s*CHF")
    matches = pattern.findall(response)
    if matches:
        min_salary, max_salary = [int(salary.replace("'", "")) for salary in matches[0]]
        return min_salary, max_salary
    return None, None


# Block 1 and Block 2 


In [11]:
# change here for new prompt and message below ------------
results_file = 'data/llm_block1_2.csv'
min = 'block1_2_min_salary'
max = 'block1_2_max_salary'
answer = 'block1_2_answer'
# Setup and load initial data
if os.path.exists(results_file):
    df_without_salary = pd.read_csv(results_file)
else:
    df[min] = None  # Initialize columns if starting fresh
    df[max] = None
    df[answer] = None 

# change content here for new prompt and message below ------------
def generate_query(row):
    return {
        "role": "user",
        "content": f"Gib mir nur das Gehalt im Format (\d{1,3}(?:'\d{3})) und (\d{1,3}(?:'\d{3})) CHF für die Position {row['title']} bei der Firma {row['company']} in Zürich, Schweiz. Wenn du kein exaktes Gehalt hast gib eine Schätzung an."
    }

def run_model(limit):
    start_index = df[min].last_valid_index() + 1 if df[max].last_valid_index() != None else 0

    
    for index, row in df.iloc[start_index:start_index + limit].iterrows():
        query_message = generate_query(row)
        chat_completion = client.chat.completions.create(
            messages= [query_message], # change message for new prompt ------------
            model="llama3-70b-8192",
            temperature=0.0
        )
        min_salary, max_salary = extract_salaries(chat_completion.choices[0].message.content)
        print(index)
        print(min_salary, max_salary)
        
        
        # Update DataFrame
        df.at[index, min] = min_salary
        df.at[index, max] = max_salary
        df.at[index, answer] = chat_completion.choices[0].message.content
        
    # Save updated DataFrame in chunks or fully, depending on your preference
    df.to_csv(results_file, index=False)

# Testing the function with a limited number of entries
run_model(10000)

0
80000 120000
1
80000 110000
2
80000 110000
3
4500 6000
4
110000 140000
5
None None
6
90000 140000
7
90000 120000
8
None None
9
90000 120000
10
120000 180000
11
90000 140000
12
120000 150000
13
80000 120000
14
120000 180000
15
120000 180000
16
120000 180000
17
80000 120000
18
80000 120000
19
80000 120000
20
90000 140000
21
80000 120000
22
80000 120000
23
90000 120000
24
None None
25
None None
26
None None
27
110000 140000
28
90000 120000
29
90000 110000
30
80000 110000
31
90000 130000
32
90000 120000
33
90000 120000
34
90000 120000
35
120000 150000
36
110000 140000
37
120000 150000
38
90000 120000
39
80000 110000
40
None None
41
90000 120000
42
None None
43
None None
44
120000 150000
45
90000 120000
46
90000 120000
47
110000 140000
48
None None
49
120000 180000
50
120000 180000
51
90000 120000
52
None None
53
None None
54
None None
55
None None
56
90000 120000
57
95000 125000
58
None None
59
None None
60
95000 125000
61
None None
62
90000 140000
63
90000 120000
64
90000 120000
65
None

OSError: Cannot save file into a non-existent directory: 'data'

Block 1 and 2 analysis


In [13]:
analysis = pd.read_csv('../../data_acquisition/data/df_with_salary.csv')

'data/llm_block1_2.csv'

# Block 1,2,3 and 4

In [ ]:
# change here for new prompt and message below ------------
results_file = 'data/llm_block1_2_3_4.csv'
min = 'block1_2_min_salary'
max = 'block1_2_max_salary'
answer = 'block1_2_answer'
# Setup and load initial data
if os.path.exists(results_file):
    df_without_salary = pd.read_csv(results_file)
else:
    df[min] = None  # Initialize columns if starting fresh
    df[max] = None
    df[answer] = None 

# change content here for new prompt and message below ------------
def generate_query(row):
    return {
        "role": "user",
        "content": f"Gib mir nur das Gehalt im Format (\d{1,3}(?:'\d{3})) und (\d{1,3}(?:'\d{3})) CHF für die Position {row['title']} bei der Firma {row['company']} in Zürich, Schweiz. Mit einer Arbeitszeit von {row['min_workload']} bis {row['max_workload']} Prozent und dem Vertragsart {row['contract_type']}. Wenn du kein exaktes Gehalt hast gib eine Schätzung an."
    }

def run_model(limit):
    start_index = df[min].last_valid_index() + 1 if df[max].last_valid_index() != None else 0

    
    for index, row in df.iloc[start_index:start_index + limit].iterrows():
        query_message = generate_query(row)
        chat_completion = client.chat.completions.create(
            messages= [query_message], # change message for new prompt ------------
            model="llama3-70b-8192",
            temperature=0.0
        )
        min_salary, max_salary = extract_salaries(chat_completion.choices[0].message.content)
        print(index)
        print(min_salary, max_salary)
        
        
        # Update DataFrame
        df.at[index, min] = min_salary
        df.at[index, max] = max_salary
        df.at[index, answer] = chat_completion.choices[0].message.content
        
    # Save updated DataFrame in chunks or fully, depending on your preference
    df.to_csv(results_file, index=False)

# Testing the function with a limited number of entries
run_model(10000)